In [ ]:
import sys
import time
import os.path
from glob import glob
from datetime import datetime, timedelta

# data tools
import h5py
import pygrib
import numpy as np
import numba as nb
import pandas as pd
import netCDF4 as nc

# custom tools
sys.path.insert(0, '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/')
sys.path.insert(0, '/glade/u/home/ksha/PUBLISH/WFRT-PP-DEV/libs/')

import utils

# !!!! <---- change to your namelist
from namelist_casper import * 
from namelist_plot import *
import plot_lib as plib

In [ ]:
BACKUP_dir = '/glade/scratch/ksha/BACKUP/'

In [ ]:
with pd.HDFStore(BACKUP_dir+'BCH_metadata.hdf', 'r') as hdf_temp:
    metadata = hdf_temp['metadata']

stn_code = metadata['code'].values
stn_lat = metadata['lat'].values
stn_lon = metadata['lon'].values

In [ ]:
# with pd.HDFStore(BACKUP_dir+'BCH_85_metadata.hdf', 'r') as hdf_temp:
#     metadata = hdf_temp['metadata']

# stn_code = metadata['code'].values
# stn_lat = metadata['lat'].values
# stn_lon = metadata['lon'].values

In [ ]:
stn_list = ['LJU', 'STA', 'COQ', 'WAH', 'CMX', 'BCK', 'ELK', 'YVR', 'YRV', 'CRS', 'ING', 'PYN', 'WON', 'YXJ']

L = len(stn_code)

In [ ]:

STN_LOCs = {}

for i in range(L):
    stn_ = stn_code[i]
    if stn_ in stn_list:
        STN_LOCs[stn_] = (stn_lon[i], stn_lat[i])

In [7]:
STN_LOCs

{'BCK': (-123.91388888888889, 48.50333333333333),
 'CMX': (-125.09444444444443, 49.643055555555556),
 'COQ': (-122.77777777777777, 49.355555555555554),
 'CRS': (-116.51666666666667, 49.1),
 'ELK': (-125.76416666666667, 49.87277777777778),
 'PYN': (-122.6375, 55.35),
 'STA': (-122.32638888888889, 49.5575),
 'WAH': (-121.61861111111111, 49.231944444444444),
 'WON': (-121.8, 56.733333333333334),
 'YRV': (-118.18333333333334, 50.96666666666667),
 'YVR': (-123.18333333333334, 49.18333333333333),
 'YXJ': (-120.73333333333333, 56.233333333333334)}

In [ ]:
LON_stn = {}
LAT_stn = {}

for i in range(L):
    stn_ = stn_code[i]
    if stn_ in stn_list:
        LON_stn[stn_] = stn_lon[i]
        LAT_stn[stn_] = stn_lat[i]

In [41]:
stn_known = list(LON_stn.keys())

In [54]:
print('| stn |    lon    |   lat   |\n|---------------------------|')

for i in range(12):
    stn_ = stn_known[i]
    print('| {} | {:.4f} | {:.4f} |'.format(stn_, LON_stn[stn_], LAT_stn[stn_]))

| stn |    lon    |   lat   |
|---------------------------|
| BCK | -123.9139 | 48.5033 |
| CMX | -125.0944 | 49.6431 |
| COQ | -122.7778 | 49.3556 |
| CRS | -116.5167 | 49.1000 |
| ELK | -125.7642 | 49.8728 |
| PYN | -122.6375 | 55.3500 |
| STA | -122.3264 | 49.5575 |
| WAH | -121.6186 | 49.2319 |
| WON | -121.8000 | 56.7333 |
| YRV | -118.1833 | 50.9667 |
| YVR | -123.1833 | 49.1833 |
| YXJ | -120.7333 | 56.2333 |


In [36]:
len(LON_stn)

12

|          | 17                  |
|:---------|:--------------------|
| code     | BCK                 |
| name     | Bear Ck. Reservoir  |
| lat      | 48.50333333333333   |
| lon      | -123.91388888888889 |
| elev [m] | 419.0               |
|          | 29                  |
|:---------|:--------------------|
| code     | CMX                 |
| name     | Comox Dam forebay   |
| lat      | 49.643055555555556  |
| lon      | -125.09444444444443 |
| elev [m] | 135.0               |
|          | 30                     |
|:---------|:-----------------------|
| code     | COQ                    |
| name     | Coquitlam Lake forebay |
| lat      | 49.355555555555554     |
| lon      | -122.77777777777777    |
| elev [m] | 160.0                  |
|          | 33                  |
|:---------|:--------------------|
| code     | CRS                 |
| name     | Creston             |
| lat      | 49.1                |
| lon      | -116.51666666666667 |
| elev [m] | 597.0               |
|          | 44                        |
|:---------|:--------------------------|
| code     | ELK                       |
| name     | Elk R. above Campbell Lk. |
| lat      | 49.87277777777778         |
| lon      | -125.76416666666667       |
| elev [m] | 270.0                     |
|          | 83        |
|:---------|:----------|
| code     | PYN       |
| name     | Pine Pass |
| lat      | 55.35     |
| lon      | -122.6375 |
| elev [m] | 1400.0    |
|          | 90                       |
|:---------|:-------------------------|
| code     | STA                      |
| name     | Stave R. above Stave Lk. |
| lat      | 49.5575                  |
| lon      | -122.32638888888889      |
| elev [m] | 330.0                    |
|          | 96                    |
|:---------|:----------------------|
| code     | WAH                   |
| name     | Wahleach (Jones) Res. |
| lat      | 49.231944444444444    |
| lon      | -121.61861111111111   |
| elev [m] | 641.0                 |
|          | 99                 |
|:---------|:-------------------|
| code     | WON                |
| name     | Wonowon            |
| lat      | 56.733333333333334 |
| lon      | -121.8             |
| elev [m] | 910.0              |
|          | 110                   |
|:---------|:----------------------|
| code     | YRV                   |
| name     | REVELSTOKE ARPT (AU5) |
| lat      | 50.96666666666667     |
| lon      | -118.18333333333334   |
| elev [m] | 443.0                 |
|          | 111                       |
|:---------|:--------------------------|
| code     | YVR                       |
| name     | VANCOUVER INTL ARPT (WS3) |
| lat      | 49.18333333333333         |
| lon      | -123.18333333333334       |
| elev [m] | 2.0                       |
|          | 113                         |
|:---------|:----------------------------|
| code     | YXJ                         |
| name     | FORT ST. JOHN ARPT (W04ATO) |
| lat      | 56.233333333333334          |
| lon      | -120.73333333333333         |
| elev [m] | 695.0                       |

In [ ]:
ELK is ECL

In [15]:
import pandas as pd

In [16]:
name_excel = '/glade/scratch/ksha/BACKUP/BCH_RAW_Files/BCH_instruments.xlsx'
pd_info = pd.read_excel(name_excel, sheet_name=None)

In [24]:
pd_info['Sheet1'].iloc[33]

HFG                                                                ECL (formerly ELK)
Halfway River above Graham River                            Elk R. above Campbell Lk.
56.50834                                                                      49.8728
-122.241                                                                     -125.764
657                                                                               270
OTT_Pluvio 2_750mm                  Standpipe_PVC_green/metalWrap ; HONEYWELL_SENS...
Name: 33, dtype: object

In [23]:
for i in range(79):
    
    print('{}-{}'.format(code_extra[i], i))

MSK-0
PKA-1
PMD-2
SKI-3
TNS-4
WCC-5
WRU-6
AKI-7
AKN-8
ALU-9
ASH-10
BAR-11
BIR-12
BLN-13
BMN-14
BRI-15
BUL-16
BVR-17
CHK-18
CHW-19
CLO-20
CLW-21
CMX-22
COQ-23
COX-24
CQM-25
DAI-26
DBC-27
DCN-28
DLU-29
DON-30
DOW-31
EAC-32
ECL (formerly ELK)-33
ERF (formerly ELR)-34
ERC-35
FDL-36
FER-37
FIN-38
FLK-39
FST-40
GLD-41
GOC-42
GOL-43
GRN-44
GRP-45
GRT-46
HEB-47
HFF-48
HRN-49
HUR-50
ILL-51
JHT-52
KEY-53
MCQ-54
MIS-55
MOB-56
MOL-57
MOR-58
MTR-59
NTY-60
PAK-61
PAR-62
PNK-63
PRS-64
PUL-65
PYN-66
QBY-67
SGL-68
SHH-69
SLK-70
STA-71
STV-72
UCE-73
WAH-74
WOL-75
WON-76
YGS-77
KWA-78
